In [1]:
import pandas as pd
import numpy as np

# all_hscode 기준으로 품목분류사례에 없는 HS 코드 100개 정답 데이터 만들기
### (기타 제외 + 10자리 보장 + 사용자 입력 변수 생성)

## 1. 데이터 로드

In [2]:
# all_hscode 로드 (기준 데이터)
all_hscode = pd.read_csv('/Users/zoohunn/Desktop/4-2/데사캡디/data/all_hscode (1).csv')
all_hscode['code'] = all_hscode['code'].astype(str)
print(f"all_hscode 전체: {len(all_hscode)}개")
all_hscode.head()

all_hscode 전체: 19406개


,code,description,parent_code,category
0,1,Animals; live,NaN,commodities
1,101,"Horses, asses, mules and hinnies; live",1.0,"horses, mules, hinnies"
2,10121,"Horses; live, pure-bred breeding animals",101.0,horses
3,101211000,농가 사육용 (For farm breeding),10121.0,말
4,101219000,기타 (Other),10121.0,말


In [3]:
# 품목분류사례 크롤링 데이터
crawling = pd.read_csv('/Users/zoohunn/Desktop/4-2/데사캡디/크롤링/품목분류사례_crawling_1005.csv')
print(f"품목분류사례 데이터: {len(crawling)}개")
crawling.head()

품목분류사례 데이터: 29961개


,번호,시행일자,HSCode,이름,설명,사유
0,1,2025-08-28,8543.70-2090,ERGOBODY SHAPY EMS BELT; EB24-SB03;,"- 전극 패드가 내장된 직물 재질의 벨트와 컨트롤러, 충전 케이블이 지제 박스에 소...",- 관세율표 제8543호에는 ‘그 밖의 전기기기(이 류에 따로 분류되지 않은 것으로...
1,2,2025-08-28,3212.90-1000,"Colouring matter, put up in forms for retail s...","o Eosin-Y dye, Acetic acid, Isopropyl alcohol,...",o 관세율표 제3212호에는 “비수성(非水性) 매질(媒質)에 분산시킨 안료[금속 가...
2,3,2025-08-28,8543.70-2090,DEHUMIDIFICATION AND AROMATHERAPY MACHINE (Lav...,"- 향기 타블렛과 제습비즈를 본체에 삽입 후 팬을 구동시켜 방향․제습하고, UV L...","- 본 물품은 팬과 UV LED 램프가 결합된 본체와 향기타블렛, 제습비즈와 USB..."
3,4,2025-08-28,3212.90-1000,"Colouring matter, put up in forms for retail s...","o Eosin-Y dye, Phosphotungstic acid hydrate, L...",o 관세율표 제3212호에는 “비수성(非水性) 매질(媒質)에 분산시킨 안료[금속 가...
4,5,2025-08-28,7007.19-1000,SODALIME GLASS,"ㅇ 물품개요\n- 게임기 모니터의 전면부에 부착되는 Cover GLASS로, 이물질...",- 관세율표 제16부 총설에서 “다른 부에 보다 구체적으로 열거한 물품을 제외한 여...


In [4]:
# 관세청 HS 부호 데이터 (한글품목명 등 추가 정보용)
df_customs = pd.read_excel('/Users/zoohunn/Desktop/4-2/데사캡디/data/관세청_HS부호_20250101 (1).xlsx')
print(f"관세청 데이터: {len(df_customs)}개")
df_customs.head()

관세청 데이터: 12467개


,HS부호,적용시작일자,적용종료일자,한글품목명,영문품목명,HS부호내용,한국표준무역분류명,수량단위최대단가,중량단위최대단가,수량단위코드,중량단위코드,수출성질코드,수입성질코드,품목규격명,필수규격명,참고규격명,규격설명,규격사항내용,성질통합분류코드,성질통합분류코드명
0,101211000,20130101,20251231,농가 사육용,For farm breeding,NaN,NaN,NaN,NaN,U,KG,11Z01,12399,NaN,NaN,NaN,NaN,NaN,11020101.0,(말)
1,101219000,20120101,20251231,기타,Other,NaN,NaN,NaN,NaN,U,KG,11Z01,12399,NaN,NaN,NaN,NaN,NaN,11020101.0,(말)
2,101291000,20120101,20251231,경주말,Horses for racing,NaN,NaN,NaN,NaN,U,KG,11Z01,12399,NaN,NaN,NaN,NaN,NaN,11020101.0,(말)
3,101299000,20120101,20251231,기타,Other,NaN,NaN,NaN,NaN,U,KG,11Z01,12399,NaN,NaN,NaN,NaN,NaN,11020101.0,(말)
4,101300000,20220101,20251231,당나귀,Asses,NaN,NaN,NaN,NaN,U,KG,11Z01,12399,NaN,NaN,NaN,NaN,NaN,11020190.0,(기타 산 동물)


## 2. 관세청 데이터에서 '기타' 아닌 코드 추출

In [5]:
# HS부호를 10자리로 정규화
df_customs['HS부호'] = df_customs['HS부호'].astype(str).str.zfill(10)

# '기타' 아닌 HS 코드만 추출
non_etc_codes = set(df_customs[
    (df_customs['한글품목명'].notna()) & 
    (df_customs['한글품목명'] != '기타')
]['HS부호'])

print(f"관세청에서 '기타' 아닌 코드: {len(non_etc_codes)}개")

관세청에서 '기타' 아닌 코드: 9350개


## 3. all_hscode에서 10자리 코드 추출

In [6]:
# 9자리 또는 10자리 코드 → 모두 10자리로 정규화
hscode_10 = all_hscode[
    all_hscode['code'].str.len().isin([9, 10]) & 
    all_hscode['code'].str.isdigit()
].copy()

# ✅ 중요: 10자리로 변환
hscode_10['code_10'] = hscode_10['code'].str.zfill(10)

print(f"10자리 코드: {len(hscode_10)}개")
print(f"10자리 확인: {hscode_10['code_10'].str.len().eq(10).all()}")
hscode_10.head()

10자리 코드: 11480개
10자리 확인: True


,code,description,parent_code,category,code_10
3,101211000,농가 사육용 (For farm breeding),10121.0,말,0101211000
4,101219000,기타 (Other),10121.0,말,0101219000
6,101291000,경주말 (Horses for racing),10129.0,말,0101291000
7,101299000,기타 (Other),10129.0,말,0101299000
9,101300000,당나귀 (Asses),10130.0,기타 산 동물,0101300000


## 4. 품목분류사례에 있는 HS 코드 추출

In [7]:
# 크롤링 데이터의 HS 코드 정리 (10자리로 정규화)
crawling['HSCode_clean'] = crawling['HSCode'].astype(str).str.replace('[.-]', '', regex=True)

# 유효한 10자리 코드만 추출
valid_codes = crawling[
    (crawling['HSCode_clean'].str.len() == 10) & 
    (crawling['HSCode_clean'].str.isdigit()) &
    (~crawling['HSCode'].astype(str).str.startswith('제'))
]['HSCode_clean'].drop_duplicates()

valid_codes_clean = set(valid_codes.str.zfill(10))
print(f"품목분류사례에 있는 유효한 HS 코드: {len(valid_codes_clean)}개")

품목분류사례에 있는 유효한 HS 코드: 3435개


## 5. 품목분류사례에 없으면서 + '기타' 아닌 코드 필터링

In [8]:
# all_hscode 10자리 중에서:
# 1) 품목분류사례에 없음
# 2) '기타' 아님
hscode_not_in_crawling = hscode_10[
    (~hscode_10['code_10'].isin(valid_codes_clean)) &
    (hscode_10['code_10'].isin(non_etc_codes))
].copy()

print(f"품목분류사례에 없고 '기타' 아닌 코드: {len(hscode_not_in_crawling)}개")
print(f"10자리 확인: {hscode_not_in_crawling['code_10'].str.len().eq(10).all()}")
hscode_not_in_crawling.head()

품목분류사례에 없고 '기타' 아닌 코드: 6712개
10자리 확인: True


,code,description,parent_code,category,code_10
3,101211000,농가 사육용 (For farm breeding),10121.0,말,0101211000
6,101291000,경주말 (Horses for racing),10129.0,말,0101291000
9,101300000,당나귀 (Asses),10130.0,기타 산 동물,0101300000
14,102211000,젖소 (For milk ),10221.0,소,0102211000
15,102212000,육우(肉牛) (For meat),10221.0,소,0102212000


## 6. 랜덤 100개 샘플링

In [9]:
# 랜덤 100개 샘플링
sampled = hscode_not_in_crawling.sample(n=min(100, len(hscode_not_in_crawling)), random_state=42)

print(f"샘플링된 코드: {len(sampled)}개")
print(f"10자리 확인: {sampled['code_10'].str.len().eq(10).all()}")
sampled[['code_10', 'description', 'category']].head(10)

샘플링된 코드: 100개
10자리 확인: True


,code_10,description,category
14355,8411821000,항공기용 (For aircraft),항공기 엔진
573,0303899020,홍살치 (Thorny head),기타 어류냉동
17872,8806210000,최대이륙중량이 250그램 이하인 것 (With maximum take-off wei...,무인기
9629,5206320000,구성하는 단사가 714.29데시텍스 미만 232.56데시텍스 이상인 것(단사당 미터...,면사
15561,8463200000,나사 전조기 (Thread rolling machines),금속성형가공기계
9823,5303901090,그 밖의 방직용 인피(靭皮)섬유 (Other textile bast fibres),마
1489,0711904000,당근 (Carrots),당근
18685,9113200000,비금속(卑金屬)으로 만든 것(금이나 은을 도금하였는지에 상관없다) (Of base ...,시계줄
8271,4302209020,라쿤(raccoon)의 것 (Of raccoon),기타모피유연처리 또는 드레스가공
13298,7403120000,와이어바(wire-bar) (Wire-bars),동


## 7. 관세청 데이터와 매칭

In [10]:
# 필요한 컬럼만 선택
customs_cols = ['HS부호', '적용시작일자', '적용종료일자', '한글품목명', '영문품목명', '성질통합분류코드명']
df_customs_subset = df_customs[customs_cols].copy()

In [11]:
# sampled와 관세청 데이터 merge
result = sampled.merge(
    df_customs_subset,
    left_on='code_10',
    right_on='HS부호',
    how='left'
)

# ✅ 중요: code_10을 HS부호로 사용 (10자리 보장)
result['HS부호'] = result['code_10']

print(f"매칭 결과: {len(result)}개")
print(f"한글품목명 있음: {result['한글품목명'].notna().sum()}개")
print(f"'기타' 개수: {(result['한글품목명'] == '기타').sum()}개")
print(f"10자리 확인: {result['HS부호'].str.len().eq(10).all()}")
result.head()

매칭 결과: 100개
한글품목명 있음: 100개
'기타' 개수: 0개
10자리 확인: True


,code,description,parent_code,category,code_10,HS부호,적용시작일자,적용종료일자,한글품목명,영문품목명,성질통합분류코드명
0,8411821000,항공기용 (For aircraft),841182.0,항공기 엔진,8411821000,8411821000,20130101,20251231,항공기용,For aircraft,(항공기 엔진)
1,303899020,홍살치 (Thorny head),30389.0,기타 어류냉동,0303899020,0303899020,20120101,20251231,홍살치,Thorny head,(기타 어류(냉동))
2,8806210000,최대이륙중량이 250그램 이하인 것 (With maximum take-off wei...,880621.0,무인기,8806210000,8806210000,20220114,20251231,최대이륙중량이 250그램 이하인 것,With maximum take-off weight not more than 250 g,(무인기)
3,5206320000,구성하는 단사가 714.29데시텍스 미만 232.56데시텍스 이상인 것(단사당 미터...,520632.0,면사,5206320000,5206320000,20220101,20251231,구성하는 단사가 714.29데시텍스 미만 232.56데시텍스 이상인 것(단사당 미터...,Measuring per single yarn less than 714.29 dec...,(면사)
4,8463200000,나사 전조기 (Thread rolling machines),846320.0,금속성형가공기계,8463200000,8463200000,20130101,20251231,나사 전조기,Thread rolling machines,(금속성형가공기계)


In [12]:
# 최종 필터링: '기타' 제외
result_filtered = result[
    result['한글품목명'].notna() & 
    (result['한글품목명'] != '기타')
].copy()

# ✅ 한 번 더 10자리 보장 (9자리면 앞에 0 추가)
result_filtered['HS부호'] = result_filtered['HS부호'].astype(str).str.zfill(10)

print(f"최종 데이터: {len(result_filtered)}개")
print(f"10자리 확인: {result_filtered['HS부호'].str.len().eq(10).all()}")
if len(result_filtered) < 100:
    print(f"⚠️ {100 - len(result_filtered)}개 부족")
else:
    print(f"✅ 100개 확보 완료")

최종 데이터: 100개
10자리 확인: True
✅ 100개 확보 완료


## 8. 4자리/6자리 부모 코드 정보 추가

In [13]:
# 4자리 코드 데이터
hscode_4 = all_hscode[all_hscode['code'].str.len().isin([3, 4])].copy()
hscode_4['code_4'] = hscode_4['code'].str.zfill(4)
hscode_4_subset = hscode_4[['code_4', 'description', 'category']].drop_duplicates(subset='code_4')
hscode_4_subset = hscode_4_subset.rename(columns={
    'description': 'description_4자리', 
    'category': 'category_4자리'
})

print(f"4자리 코드: {len(hscode_4_subset)}개")

4자리 코드: 1229개


In [14]:
# 6자리 코드 데이터
hscode_6 = all_hscode[all_hscode['code'].str.len().isin([5, 6])].copy()
hscode_6['code_6'] = hscode_6['code'].str.zfill(6)
hscode_6_subset = hscode_6[['code_6', 'description', 'category']].drop_duplicates(subset='code_6')
hscode_6_subset = hscode_6_subset.rename(columns={
    'description': 'description_6자리', 
    'category': 'category_6자리'
})

print(f"6자리 코드: {len(hscode_6_subset)}개")

6자리 코드: 5619개


In [15]:
# 4자리, 6자리 추출
result_filtered['HS_4자리'] = result_filtered['HS부호'].str[:4]
result_filtered['HS_6자리'] = result_filtered['HS부호'].str[:6]

In [16]:
# merge
result_filtered = result_filtered.merge(hscode_4_subset, left_on='HS_4자리', right_on='code_4', how='left')
result_filtered = result_filtered.merge(hscode_6_subset, left_on='HS_6자리', right_on='code_6', how='left')

# 불필요한 컬럼 제거
cols_to_drop = ['code', 'code_10', 'code_4', 'code_6', 'description', 'category']
result_filtered = result_filtered.drop([c for c in cols_to_drop if c in result_filtered.columns], axis=1)

result_filtered.head()

,parent_code,HS부호,적용시작일자,적용종료일자,한글품목명,영문품목명,성질통합분류코드명,HS_4자리,HS_6자리,description_4자리,category_4자리,description_6자리,category_6자리
0,841182.0,8411821000,20130101,20251231,항공기용,For aircraft,(항공기 엔진),8411,841182,"Turbo-jets, turbo-propellers and other gas tur...","propellers, jets, machinery, nuclear, reactors...",Turbines; gas-turbines (excluding turbo-jets a...,turbi
1,30389.0,0303899020,20120101,20251231,홍살치,Thorny head,(기타 어류(냉동)),0303,030389,"Fish; frozen, excluding fish fillets and other...",fish,"Fish; frozen, n.e.c. in heading 0303, excludin...",fish
2,880621.0,8806210000,20220114,20251231,최대이륙중량이 250그램 이하인 것,With maximum take-off weight not more than 250 g,(무인기),8806,880621,Unmanned aircraft,"unmanned, aircraft",Unmanned aircraft; for remote-controlled fligh...,"unmanned, aircraft"
3,520632.0,5206320000,20220101,20251231,구성하는 단사가 714.29데시텍스 미만 232.56데시텍스 이상인 것(단사당 미터...,Measuring per single yarn less than 714.29 dec...,(면사),5206,520632,"Cotton yarn (other than sewing thread), contai...","yarn, cotton","Cotton yarn; (not sewing thread), multiple or ...","yarn, cotton"
4,846320.0,8463200000,20130101,20251231,나사 전조기,Thread rolling machines,(금속성형가공기계),8463,846320,"Machine-tools; n.e.c. for working metal, sinte...","machine, tools","Machine-tools; for working metal, sintered met...","machine, tools"


## 9. 원본 데이터 저장

In [17]:
# 컬럼 순서 정리
col_order = [
    'HS부호', '적용시작일자', '적용종료일자', '한글품목명', '영문품목명', '성질통합분류코드명',
    'HS_4자리', 'HS_6자리', 'description_4자리', 'category_4자리', 'description_6자리', 'category_6자리'
]
result_final = result_filtered[[c for c in col_order if c in result_filtered.columns]].copy()

# ✅ 최종 10자리 보장 (저장 전)
result_final['HS부호'] = result_final['HS부호'].astype(str).str.zfill(10)

print(f"최종 데이터: {len(result_final)}개")
print(f"10자리 확인: {result_final['HS부호'].str.len().eq(10).all()}")
print("컬럼:", result_final.columns.tolist())
result_final.head()

최종 데이터: 100개
10자리 확인: True
컬럼: ['HS부호', '적용시작일자', '적용종료일자', '한글품목명', '영문품목명', '성질통합분류코드명', 'HS_4자리', 'HS_6자리', 'description_4자리', 'category_4자리', 'description_6자리', 'category_6자리']


,HS부호,적용시작일자,적용종료일자,한글품목명,영문품목명,성질통합분류코드명,HS_4자리,HS_6자리,description_4자리,category_4자리,description_6자리,category_6자리
0,8411821000,20130101,20251231,항공기용,For aircraft,(항공기 엔진),8411,841182,"Turbo-jets, turbo-propellers and other gas tur...","propellers, jets, machinery, nuclear, reactors...",Turbines; gas-turbines (excluding turbo-jets a...,turbi
1,0303899020,20120101,20251231,홍살치,Thorny head,(기타 어류(냉동)),0303,030389,"Fish; frozen, excluding fish fillets and other...",fish,"Fish; frozen, n.e.c. in heading 0303, excludin...",fish
2,8806210000,20220114,20251231,최대이륙중량이 250그램 이하인 것,With maximum take-off weight not more than 250 g,(무인기),8806,880621,Unmanned aircraft,"unmanned, aircraft",Unmanned aircraft; for remote-controlled fligh...,"unmanned, aircraft"
3,5206320000,20220101,20251231,구성하는 단사가 714.29데시텍스 미만 232.56데시텍스 이상인 것(단사당 미터...,Measuring per single yarn less than 714.29 dec...,(면사),5206,520632,"Cotton yarn (other than sewing thread), contai...","yarn, cotton","Cotton yarn; (not sewing thread), multiple or ...","yarn, cotton"
4,8463200000,20130101,20251231,나사 전조기,Thread rolling machines,(금속성형가공기계),8463,846320,"Machine-tools; n.e.c. for working metal, sinte...","machine, tools","Machine-tools; for working metal, sintered met...","machine, tools"


In [18]:
# CSV 저장
result_final.to_csv('/Users/zoohunn/Desktop/4-2/데사캡디/data/품목분류사례없는_100sample_원본.csv', index=False)
print("✅ 저장 완료: 품목분류사례없는_100sample_원본.csv")

✅ 저장 완료: 품목분류사례없는_100sample_원본.csv


## 10. 사용자 입력 변수 생성

In [19]:
# ✅ 중요: result_final 복사본에서 작업
data = result_final.copy()

# 1. 사용자_상품명 = 한글품목명
data['사용자_상품명'] = data['한글품목명']

# 2. 사용자_상품설명 = 성질통합분류코드명 + 대분류 + 중분류
data['사용자_상품설명'] = (
    data['성질통합분류코드명'].fillna('').astype(str) + 
    ' 대분류: ' + 
    data['description_4자리'].fillna('').astype(str) + 
    ', 중분류: ' + 
    data['description_6자리'].fillna('').astype(str)
).str.strip()

# ✅ 최종 10자리 보장
data['HS부호'] = data['HS부호'].astype(str).str.zfill(10)

print(f"사용자 입력 변수 추가 완료: {len(data)}개")
print(f"10자리 확인: {data['HS부호'].str.len().eq(10).all()}")
data.head()

사용자 입력 변수 추가 완료: 100개
10자리 확인: True


,HS부호,적용시작일자,적용종료일자,한글품목명,영문품목명,성질통합분류코드명,HS_4자리,HS_6자리,description_4자리,category_4자리,description_6자리,category_6자리,사용자_상품명,사용자_상품설명
0,8411821000,20130101,20251231,항공기용,For aircraft,(항공기 엔진),8411,841182,"Turbo-jets, turbo-propellers and other gas tur...","propellers, jets, machinery, nuclear, reactors...",Turbines; gas-turbines (excluding turbo-jets a...,turbi,항공기용,"(항공기 엔진) 대분류: Turbo-jets, turbo-propellers and..."
1,0303899020,20120101,20251231,홍살치,Thorny head,(기타 어류(냉동)),0303,030389,"Fish; frozen, excluding fish fillets and other...",fish,"Fish; frozen, n.e.c. in heading 0303, excludin...",fish,홍살치,"(기타 어류(냉동)) 대분류: Fish; frozen, excluding fish ..."
2,8806210000,20220114,20251231,최대이륙중량이 250그램 이하인 것,With maximum take-off weight not more than 250 g,(무인기),8806,880621,Unmanned aircraft,"unmanned, aircraft",Unmanned aircraft; for remote-controlled fligh...,"unmanned, aircraft",최대이륙중량이 250그램 이하인 것,"(무인기) 대분류: Unmanned aircraft, 중분류: Unmanned ai..."
3,5206320000,20220101,20251231,구성하는 단사가 714.29데시텍스 미만 232.56데시텍스 이상인 것(단사당 미터...,Measuring per single yarn less than 714.29 dec...,(면사),5206,520632,"Cotton yarn (other than sewing thread), contai...","yarn, cotton","Cotton yarn; (not sewing thread), multiple or ...","yarn, cotton",구성하는 단사가 714.29데시텍스 미만 232.56데시텍스 이상인 것(단사당 미터...,(면사) 대분류: Cotton yarn (other than sewing threa...
4,8463200000,20130101,20251231,나사 전조기,Thread rolling machines,(금속성형가공기계),8463,846320,"Machine-tools; n.e.c. for working metal, sinte...","machine, tools","Machine-tools; for working metal, sintered met...","machine, tools",나사 전조기,(금속성형가공기계) 대분류: Machine-tools; n.e.c. for work...


In [20]:
# 샘플 확인
print("=== 사용자 입력 샘플 ===")
for i in range(min(5, len(data))):
    print(f"\n[{i+1}]")
    print(f"HS부호: {data.iloc[i]['HS부호']}")
    print(f"사용자_상품명: {data.iloc[i]['사용자_상품명']}")
    print(f"사용자_상품설명: {data.iloc[i]['사용자_상품설명'][:120]}...")

=== 사용자 입력 샘플 ===

[1]
HS부호: 8411821000
사용자_상품명: 항공기용
사용자_상품설명: (항공기 엔진) 대분류: Turbo-jets, turbo-propellers and other gas turbines, 중분류: Turbines; gas-turbines (excluding turbo-jets and...

[2]
HS부호: 0303899020
사용자_상품명: 홍살치
사용자_상품설명: (기타 어류(냉동)) 대분류: Fish; frozen, excluding fish fillets and other fish meat of heading 0304, 중분류: Fish; frozen, n.e.c. in ...

[3]
HS부호: 8806210000
사용자_상품명: 최대이륙중량이 250그램 이하인 것
사용자_상품설명: (무인기) 대분류: Unmanned aircraft, 중분류: Unmanned aircraft; for remote-controlled flight only, for other than for carriage of ...

[4]
HS부호: 5206320000
사용자_상품명: 구성하는 단사가 714.29데시텍스 미만 232.56데시텍스 이상인 것(단사당 미터식 번수 14수 초과 43수 이하)
사용자_상품설명: (면사) 대분류: Cotton yarn (other than sewing thread), containing less than 85% by weight of cotton, not put up for retail sa...

[5]
HS부호: 8463200000
사용자_상품명: 나사 전조기
사용자_상품설명: (금속성형가공기계) 대분류: Machine-tools; n.e.c. for working metal, sintered metal carbides or cermets without removing material, 중...


## 11. 최종 데이터 저장

In [21]:
# CSV 저장
data.to_csv('/Users/zoohunn/Desktop/4-2/데사캡디/data/품목분류사례없는_100sample_1103.csv', index=False)
print("✅ 최종 저장 완료: 품목분류사례없는_100sample_1103.csv")
print(f"총 {len(data)}개 샘플")
print(f"10자리 확인: {data['HS부호'].str.len().eq(10).all()}")
print(f"\n포함된 컬럼: {data.columns.tolist()}")

✅ 최종 저장 완료: 품목분류사례없는_100sample_1103.csv
총 100개 샘플
10자리 확인: True

포함된 컬럼: ['HS부호', '적용시작일자', '적용종료일자', '한글품목명', '영문품목명', '성질통합분류코드명', 'HS_4자리', 'HS_6자리', 'description_4자리', 'category_4자리', 'description_6자리', 'category_6자리', '사용자_상품명', '사용자_상품설명']


## 12. 최종 검증

In [22]:
# 필수 컬럼 확인
required_cols = ['HS부호', '사용자_상품명', '사용자_상품설명', '한글품목명']
print("=== 필수 컬럼 확인 ===")
for col in required_cols:
    exists = col in data.columns
    if exists:
        null_count = data[col].isna().sum()
        print(f"  {col}: ✅ (결측: {null_count})")
    else:
        print(f"  {col}: ❌ 없음!")

=== 필수 컬럼 확인 ===
  HS부호: ✅ (결측: 0)
  사용자_상품명: ✅ (결측: 0)
  사용자_상품설명: ✅ (결측: 0)
  한글품목명: ✅ (결측: 0)


In [23]:
# HS부호 길이 분포 확인
print("\n=== HS부호 길이 분포 ===")
length_dist = data['HS부호'].astype(str).str.len().value_counts().sort_index()
print(length_dist)
if (data['HS부호'].astype(str).str.len() == 10).all():
    print("✅ 모든 코드가 10자리")
else:
    print("⚠️ 경고: 10자리 아닌 코드 있음")
    non_10 = data[data['HS부호'].astype(str).str.len() != 10]
    print(non_10[['HS부호', '한글품목명']].head())


=== HS부호 길이 분포 ===
HS부호
10    100
Name: count, dtype: int64
✅ 모든 코드가 10자리


In [24]:
# 품목분류사례 중복 확인
print("\n=== 품목분류사례 중복 확인 ===")
overlap = data[data['HS부호'].isin(valid_codes_clean)]
print(f"품목분류사례와 중복: {len(overlap)}개")
if len(overlap) > 0:
    print("⚠️ 중복 코드:", overlap['HS부호'].tolist())
else:
    print("✅ 중복 없음")


=== 품목분류사례 중복 확인 ===
품목분류사례와 중복: 0개
✅ 중복 없음


In [25]:
# '기타' 확인
print("\n=== '기타' 확인 ===")
etc_count = (data['한글품목명'] == '기타').sum()
print(f"'기타' 개수: {etc_count}개")
if etc_count > 0:
    print("⚠️ '기타' 포함됨")
else:
    print("✅ '기타' 없음")


=== '기타' 확인 ===
'기타' 개수: 0개
✅ '기타' 없음


In [26]:
# 최종 통계
print("\n=== 최종 통계 ===")
print(f"총 샘플 수: {len(data)}개")
print(f"HS부호 고유값: {data['HS부호'].nunique()}개")
print(f"사용자_상품명 결측: {data['사용자_상품명'].isna().sum()}개")
print(f"사용자_상품설명 결측: {data['사용자_상품설명'].isna().sum()}개")
print(f"\n전체 컬럼 ({len(data.columns)}개):")
print(data.columns.tolist())


=== 최종 통계 ===
총 샘플 수: 100개
HS부호 고유값: 100개
사용자_상품명 결측: 0개
사용자_상품설명 결측: 0개

전체 컬럼 (14개):
['HS부호', '적용시작일자', '적용종료일자', '한글품목명', '영문품목명', '성질통합분류코드명', 'HS_4자리', 'HS_6자리', 'description_4자리', 'category_4자리', 'description_6자리', 'category_6자리', '사용자_상품명', '사용자_상품설명']
